<a href="https://colab.research.google.com/github/astrodatos/proyecto_iluminatiuwu/blob/master/Algoritmo_(beta).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits #importamos fits aunque no creemos que lo usaremos
from astropy.wcs import WCS #importamos WCS aunque no creemos que lo usaremos
!pip install astroquery #esto para instalar astroquery
from astroquery.gaia import Gaia 

     |████████████████████████████████| 4.2MB 2.8MB/s 
     |████████████████████████████████| 2.3MB 32.4MB/s 
     |████████████████████████████████| 61kB 22.5MB/s 
     |████████████████████████████████| 102kB 27.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/8a/d1/1e/4124d9ef35e2bbfbaa284c97dd49d9babbab42d966c4bea190
Successfully built astroquery
Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [16]:
job = Gaia.launch_job_async("select top 100000 \
solution_id,random_index,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_val,lum_val \
from gaiadr2.gaia_source order by random_index") #obtenemos los datos de 1000000 estrellas con los anteriores atributos en un Asynchronous query

print(job) 
r = job.get_results()
print(r)

Query finished.
<Table masked=True length=100000>
      name        dtype  unit format                            description                             n_bad
---------------- ------- ---- ------ ------------------------------------------------------------------ -----
     solution_id   int64                                                            Solution Identifier     0
    random_index   int64                                            Random index used to select subsets     0
       source_id   int64             Unique source identifier (unique within a particular Data Release)     0
              ra float64  deg {!r:>}                                                    Right ascension     0
             dec float64  deg {!r:>}                                                        Declination     0
 phot_g_mean_mag float32  mag {!r:>}                                              G-band mean magnitude     0
phot_bp_mean_mag float32  mag {!r:>}                                  

In [17]:
jobpd=r.to_pandas() #convertimos el job a pandas
jobpd[:10] #vemos las primeras 10 filas para ver que todo está bien

,solution_id,random_index,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_val,lum_val
0,1635721458409799680,0,4205572950572486656,284.997397,-7.213794,18.246519,18.827530,17.534872,1.292658,0.581011,0.711647,NaN,NaN
1,1635721458409799680,1,4205565597589384320,285.403309,-7.148584,17.277395,17.743320,16.645607,1.097713,0.465925,0.631788,NaN,NaN
2,1635721458409799680,2,4053206649305182080,275.626769,-25.133812,19.644129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1635721458409799680,3,4205568964799943680,285.346380,-7.110390,20.402403,20.705967,19.725151,0.980816,0.303564,0.677252,NaN,NaN
4,1635721458409799680,4,4049398731303340416,274.009409,-30.525125,19.241173,19.158228,18.095499,1.062729,-0.082945,1.145674,NaN,NaN
5,1635721458409799680,5,4053206924184153472,275.737952,-25.143911,19.238544,19.359943,17.496092,1.863852,0.121399,1.742453,NaN,NaN
6,1635721458409799680,6,2944217403015932032,96.875156,-15.996720,21.065298,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1635721458409799680,7,4205572263340546688,284.957509,-7.214939,20.697376,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1635721458409799680,8,5889059805439854848,234.271140,-52.218658,16.638042,17.331743,15.816595,1.515148,0.693701,0.821447,4368.459961,NaN
9,1635721458409799680,9,4049398765669294976,274.018619,-30.513203,19.936489,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pd.Series.count(jobpd.iloc[:,11]) #con esto obtenemos el número de estrellas que poseen info de teff_val
datamask = np.logical_not(np.isnan(jobpd.iloc[:,11]))

filtdata=jobpd.loc[datamask]

filtdata['class'] = 'A' #esto se arregla en la celda de abajo

filtdata

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,solution_id,random_index,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_val,lum_val,class
8,1635721458409799680,8,5889059805439854848,234.271140,-52.218658,16.638042,17.331743,15.816595,1.515148,0.693701,0.821447,4368.459961,NaN,A
12,1635721458409799680,12,2026240608225775360,293.518842,28.533663,16.985237,17.438818,16.372440,1.066378,0.453581,0.612797,5070.041992,NaN,A
17,1635721458409799680,17,5889050803187502976,233.816075,-52.261231,16.839481,17.559883,15.989998,1.569885,0.720402,0.849483,4361.000000,NaN,A
21,1635721458409799680,21,1870129370301798784,310.979513,34.486223,16.415411,16.938528,15.721798,1.216730,0.523117,0.693613,4919.000000,0.704140,A
24,1635721458409799680,24,5543541609247095168,127.206923,-34.350520,16.833437,17.228180,16.272953,0.955227,0.394743,0.560484,5364.750000,NaN,A
29,1635721458409799680,29,2022160423644954880,290.232745,22.548845,15.774138,16.889832,14.717401,2.172431,1.115693,1.056738,3610.000000,NaN,A
31,1635721458409799680,31,4205570785908969984,284.920012,-7.280468,13.914678,14.763895,13.013571,1.750324,0.849217,0.901107,3987.000000,28.196486,A
44,1635721458409799680,44,5435526445919095424,149.461823,-33.987084,15.605809,15.907345,15.137931,0.769414,0.301536,0.467878,6000.000000,1.463973,A
55,1635721458409799680,55,2944247433426596096,93.991029,-17.959406,16.735666,17.250914,16.071472,1.179441,0.515247,0.664194,4900.500000,NaN,A
85,1635721458409799680,85,4655038013888170752,75.692188,-70.738686,16.767576,17.608015,15.850858,1.757157,0.840439,0.916718,3989.666748,NaN,A


In [20]:
maskO = (filtdata["teff_val"]>=33000)
filtdata.loc[maskO,'class'] = 'O'
maskB = (filtdata["teff_val"]>=10000) & (filtdata["teff_val"]<33000)
filtdata.loc[maskB,'class'] = 'B'
maskA = (filtdata["teff_val"]>=7500) & (filtdata["teff_val"]<10000)
filtdata.loc[maskA,'class'] = 'A'
maskF = (filtdata["teff_val"]>=6000) & (filtdata["teff_val"]<7500)
filtdata.loc[maskF,'class'] = 'F'
maskG = (filtdata["teff_val"]>=5200) & (filtdata["teff_val"]<6000)
filtdata.loc[maskG,'class'] = 'G'
maskK = (filtdata["teff_val"]>=3700) & (filtdata["teff_val"]<5200)
filtdata.loc[maskK,'class'] = 'K'
maskM = (filtdata["teff_val"]<3700)
filtdata.loc[maskM,'class'] = 'M'

filtdata

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,solution_id,random_index,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_val,lum_val,class
8,1635721458409799680,8,5889059805439854848,234.271140,-52.218658,16.638042,17.331743,15.816595,1.515148,0.693701,0.821447,4368.459961,NaN,K
12,1635721458409799680,12,2026240608225775360,293.518842,28.533663,16.985237,17.438818,16.372440,1.066378,0.453581,0.612797,5070.041992,NaN,K
17,1635721458409799680,17,5889050803187502976,233.816075,-52.261231,16.839481,17.559883,15.989998,1.569885,0.720402,0.849483,4361.000000,NaN,K
21,1635721458409799680,21,1870129370301798784,310.979513,34.486223,16.415411,16.938528,15.721798,1.216730,0.523117,0.693613,4919.000000,0.704140,K
24,1635721458409799680,24,5543541609247095168,127.206923,-34.350520,16.833437,17.228180,16.272953,0.955227,0.394743,0.560484,5364.750000,NaN,G
29,1635721458409799680,29,2022160423644954880,290.232745,22.548845,15.774138,16.889832,14.717401,2.172431,1.115693,1.056738,3610.000000,NaN,M
31,1635721458409799680,31,4205570785908969984,284.920012,-7.280468,13.914678,14.763895,13.013571,1.750324,0.849217,0.901107,3987.000000,28.196486,K
44,1635721458409799680,44,5435526445919095424,149.461823,-33.987084,15.605809,15.907345,15.137931,0.769414,0.301536,0.467878,6000.000000,1.463973,F
55,1635721458409799680,55,2944247433426596096,93.991029,-17.959406,16.735666,17.250914,16.071472,1.179441,0.515247,0.664194,4900.500000,NaN,K
85,1635721458409799680,85,4655038013888170752,75.692188,-70.738686,16.767576,17.608015,15.850858,1.757157,0.840439,0.916718,3989.666748,NaN,K


In [0]:
filtdata.to_csv("tabla_temperatura_100mil.csv")

In [0]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier  

